# Exercise sheet 2

__Handout date:__ 23.05.2024  
__Submission deadline:__ 19.06.2024 - 23:59  
__Topics:__ Classification.  
__Submission link:__ https://fz-juelich.sciebo.de/s/qwm3ipclLcnfyOo

Add your answers by editing the marked cells. Keep your answers brief and clear.

## MNIST - Dataset

-- __Task__: Load the MNIST-dataset from scikit-learn

-- __Task__: Split the dataset into train- (70%), validation- (20%) and test-dataset (10%) (with [sklearn - train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html))

-- __Task__: Visualize the first training example of each class

## kNN - Classification

There are two hyperparameters for kNN-Classification: the distance metric and k. In this exercise we will use the Manhattan distance (L1), and will find the optimal k.

-- __Question__: Name one advantage and one disadvantage of kNN-Classification.

__Answer__:

-- __Task__: Choose k=1. Calculate a kNN model with k=1 using the scikit-learn package. Only use the training dataset for the model.

-- __Task__: Calculate the prediction of this model for the validation datset.

-- __Task__: Repeat these steps for k=1 to 10.

-- __Task__: Plot the accuracy for k=1 to 10. Choose the best k.

-- __Task__: Train a new kNN model with your chosen k and the train- and validation-dataset.

-- __Task__: Evaluate the final model on the test-dataset.

-- __Task__: Plot a [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) of the model on the test-dataset.

-- __Question__: Which two classes are confused the most? Does this results surprise you?

__Answer__:

## PCA and linear classification

In these next tasks we want to compute simple features and use them to train a linear classifier.

--__Task__: Calculate a [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for the training dataset.

-- __Task__: Choose as many principal components as needed to reach 90% explained variance.

-- __Task__: Train a [linear classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html#sklearn.linear_model.Perceptron) with these principal components.

-- __Task__: Compute the accuracy for this model on the test-dataset.

## MLP classification

Training a CNN neural network is too complex for a exercise sheet, but we will revisit them for our next project. In this exercise we will train a multi-layer-perceptron for the classification task on the MNIST-dataset.

-- __Task__: Normalize the training data to `mean=0` and `std=1`.

-- __Task__: Fit an [MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier) with appropriate hyperparameters.

-- __Task__: Apply the model to the test dataset (don't forget to normalize the test dataset).

-- __Task__: Calculate the confusion matrix for the test dataset.

## Comparison

-- __Task__: Plot the accuracy of all three models (kNN, PCA + Linear and MLP).

-- __Task__: Choose the best model and think of a possible improvement.